# JASPAR DB PARSING

-Jaspar SQL Dump was downloaded from http://jaspar.genereg.net/downloads/ .

-Built the database on my computer using HeidiSQL (MySQL style DB)

-Used PyMySQL and Biopython to access my local copy of the database and pull relevant information, which I compiled into a dictionary.

-Saved this dictionary as a JSON file for future use.


In [ ]:
import sys
!{sys.executable} -m pip install mysqlclient pymysql
!{sys.executable} -m pip install beautifulsoup4
#anaconda is too good

In [ ]:
from Bio.motifs.jaspar.db import JASPAR5

JASPAR_DB_HOST = "127.0.0.1" #provided it's on your computer
JASPAR_DB_NAME = 'jaspar'
JASPAR_DB_USER = "root" 
JASPAR_DB_PASS = 'tomato1393'
jdb = JASPAR5(
     host=JASPAR_DB_HOST,
     name=JASPAR_DB_NAME,
     user=JASPAR_DB_USER,
     password=JASPAR_DB_PASS
)

motifs=jdb.fetch_motifs()

-Creating the dictionary below, the format is:
{protein_name: {'class':class,'family':family,'species':species,'acc' = UniProt ID(s),'pubmed':reference, 'motif': motif in pfm format}
Thus, each protein has a dictionary containing its attributes as its value, and the proteins are keys in the overall dictionary.

In [ ]:
jaspardict = {}

for motif in motifs:
    jaspardict[motif.name]={'class':motif.tf_class,'family':motif.tf_family,'species':motif.species, 'acc':motif.acc,'pubmed':motif.medline, 'motif':motif.format("pfm")}



-The biopython package gives uniprot taxonomical IDs instead of species names, so for future use I'm adding in the species name as a value to the species key for each tf (but keeping the taxonomical id). The code in the cell below searches the taxonomical id on uniprot and gets the title of the page, which is the species name.

In [ ]:
import sys,requests
from bs4 import BeautifulSoup as soup

query_start = 'https://www.uniprot.org/taxonomy/'
species_ids = []
for tf in jaspardict.values():
    if tf['species']:
        spec_id = tf['species'][0]
    if spec_id not in species_ids:
        species_ids.append(spec_id)
id_to_species = {}
for spec_id in species_ids:
    r = requests.get(query_start+spec_id+"/")
    s = soup(r.text,'html.parser')
    s.prettify()
    id_to_species[spec_id]=s.title.string
print (id_to_species)
        


In [ ]:
for tf in jaspardict.values():
    if tf['species']:
        tf['species'] = str(tf['species'])+" "+id_to_species[tf['species'][0]]


Below is the code to extract the coordinates of the DNA binding domains for each of the tfs from uniprot, which is complicated by the fact that this information is stored in two different places on the web page for the protein, and also that several of the dna binding domain names are labelled in a few different ways (eg. HTH, Helix-turn-helix, H-T-H). However, the cell below will extract the DNA binding domains for almost all tfs which have annotated DBDs in uniprot (1198/1392). Might go into pfam at some point to get the rest if possible. The cell below will take ~45 min to run. 

In [ ]:
import re
import lxml
query_start = 'http://www.uniprot.org/uniprot/'
query_end  = '#family_and_domains'
domain_names = re.compile('bHLH|bZIP|Homeo|HTH|WH|wHTH|HMG|MADS|BZIP|Helix|Leucine|OB|Wor3|Dof|RHD|C2H2|H15|Paired|ETS|GATA|ARID|Zinc$|DNA$|DNA-binding|TCP|H-T-H|')
n = re.compile('DNA binding|Zinc')
count = 0
for tf in jaspardict.values():
    tf['domain']=[]
    if tf['acc']:
        uniprot_id = tf['acc'][0]
        r = requests.get(query_start+uniprot_id.rstrip())
        if r.status_code == 200:
            try:
                s = soup(r.text, 'html.parser')
                s.prettify()
                fam_domain = s.find(id='domainsAnno_section')
                region = s.find(id='regionAnno_section')
                if region:
                    x = region.find_all('td')
                    for i,t in enumerate(x):
                        if re.search(n,str(t.span)):
                            tf['domain'].append(str(x[i+1].text))
                if fam_domain:
                    if not tf['domain']:
                        x= fam_domain.find_all('td')
                        for i,t in enumerate(x):
                            if len(t.attrs):
                                if 'featdescription' in t['class']:
                                    if re.search(domain_names,t.span.text):
                                        tf['domain'].append(str(x[i-1].text))
            except:
                continue


Download the uniprot fasta dump from http://www.uniprot.org/downloads. The code below builds a dictionary mapping the uniprot protein ID to its sequence.

In [ ]:
from Bio import SeqIO
seqdict = {}
with open('uniprot_sprot.fasta/uniprot_sprot.fasta','r') as seqs:
    for seq in SeqIO.parse(seqs, "fasta"):
        seqdict[seq.id.split('|')[1]] =str(seq.seq)

The code below adds the DNA binding domains to the domains key for each protein.

In [ ]:
for tf in jaspardict.values():
    if tf['acc']:
        accession= tf['acc'][0]
        if accession in seqdict:
            if tf['domain']:
                tf['domains'] = []
                for j in tf['domain']:
                    indices = re.findall('[0-9]+',str(j))
                    if len(indices)==2:
                        dbd = seqdict[accession][int(indices[0])-1:int(indices[1])]
                        tf['domains'] = tf['domains']+[dbd]


Count of TFs without DBDs

In [ ]:
count = 0
for tf in jaspardict.values():
    if tf['acc'] and not tf['domain']:
        print (tf['acc'])
        count+=1
print(count)

Remove domain key

In [ ]:
for tf in jaspardict.values():
    del tf['domain']
    

Add empty list to domains key for tfs without DBD so I don't get KeyErrors

In [ ]:
for tf in jaspardict.values():
    if 'domains'  not in tf
        tf['domains']=[]


Dumps the jaspar dictionary to a json file. Optionally will also dump the uniprot dictionary, for future fast look up.

In [ ]:
import json
j=json.dumps(jaspardict, indent =4)
with open('jaspar.json','w') as f:
    print(j, file = f)
#with open('uniprot.json','w') as u:
#    print(seqdict, file = u)